## Concatenating all attention distributions without windowing and no indexing into the attention distribution

We extract the attention distribution directly from a BERT model that has not been pretrained for bias detection. Also we do not allow ourselves to select the attention distribution by indexing into the one that we predict is biased - because that defeates the whole point anyway. 

In [3]:
import sys; sys.path.append("../../../../..")
import torch 
from src.experiment import AttentionExperiment, ClassificationExperiment
from src.dataset import ExperimentDataset
from src.params import Params
from src.utils.attention_utils import reduce_attention_dist, return_idx_attention_dist, window_attention_dist
from src.utils.classification_utils import run_bootstrapping
from src.utils.shared_utils import get_bias_predictions

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
params = Params.read_params("linear-params.json")
print("model = {}".format(params.final_task['model']))
print("layers = {}".format(params.intermediary_task["attention"]["layers"]))
print("reducer = {}".format(params.intermediary_task["attention"]["reducer"]))

model = shallow_nn
layers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
reducer = concat


In [6]:
# Loading in the dataset that we are using in this experiments 
# typically this dataset is the small set of ground-truth labels
dataset = ExperimentDataset.init_dataset(params.dataset)

04/09/2020 07:57:55 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
386it [00:00, 4622.87it/s]


### Attention Experiment: 
* Is a class that wraps useful methods to extract attention distributions from a given BERT-based model 
* The user has to provide in two config files: One to specify parameters for how the attention scores should be extracted and combined, and other to specify the intermediary model from which the attention scores should be extracted from
* The user needs to instantiate the attention experiment with a function that tells the model how to run 
 inference on the given model. The function header is specified below: 
 
 ``` def initialize_attention_experiment(cls, intermediary_task_params, dataset_params, verbose=False) ```
 


In [7]:
attention_dataloader = dataset.return_dataloader(batch_size=params.intermediary_task['attention']['attention_extraction_batch_size']) 
attention_experiment = AttentionExperiment.initialize_attention_experiment(params.intermediary_task, params.dataset, verbose=True, from_pretrained=False)

04/09/2020 07:57:56 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
04/09/2020 07:57:56 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
04/09/2020 07:57:56 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpajmoimlw
04/09/2020 07:58:00 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_d

Instantiated joint model with default HuggingFace weights.
Succesfully loaded in attention experiment!


```extract_attention_scores()``` works out of the box because the attention experiment has the config file saved, and knows what BERT model to use/load in, which layers to extract the attention scores from, and what the inference function is that should be used on this particular BERT model.

Attention_scores is then a list of dictionaries. The keys in this dictionary are the specific layers of a BERT model and the values are the corresponding attention distributions extracted from that particular layer.

In [32]:
attention_scores = attention_experiment.extract_attention_scores(attention_dataloader)

Averaging accross the attention distributions

In [28]:
from src.utils.attention_utils import avg_attention_dist_accross_tokens

In [33]:
attention_scores_avg = avg_attention_dist_accross_tokens(attention_scores)

In [35]:
reduced_attention = reduce_attention_dist(attention_scores_avg, params.intermediary_task["attention"])
attention_dist = reduced_attention

In [36]:
attention_dist.shape

torch.Size([324, 960])

In [37]:
dataset.add_data(attention_dist, "attention_dist")
dataset.shuffle_data()
assert(attention_dist.shape[1] == params.final_task['input_dim'])

### This is where the classification experiment starts

We create a classification experiment that contains useful methods for classifying bias based on the attention distributions. 

In [38]:
params = Params.read_params("linear-params.json")

In [39]:
classification_experiment = ClassificationExperiment.init_cls_experiment(params.final_task)

In [40]:
stats = run_bootstrapping(classification_experiment, dataset, params.final_task, overfit_test=False, shuffle_data=True, num_bootstrap_iters=10, input_key="attention_dist", label_key="bias_label")

/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 

/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 

/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 

/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 

/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 

In [41]:
params.final_task

{'model': 'shallow_nn',
 'dropout_prob': 0,
 'input_dim': 960,
 'hidden_dim': 15,
 'output_dim': 1,
 'data_split': {'train_split': 0.8, 'eval_split': 0.2, 'test_split': 0},
 'training_params': {'optimizer': 'adam',
  'loss': 'bce_with_logits',
  'num_epochs': 120,
  'batch_size': 32,
  'lr': 0.0005}}

In [42]:
stats

{'auc': [(0.5153855519480519, 0.6434740259740259), 0.5828080099107334],
 'accuracy': [(0.53828125, 0.748046875), 0.625],
 'f1': [(0.6962028985507247, 0.8520754716981133), 0.7634671435978772],
 'precision': [(0.5377016129032257, 0.7587900641025641), 0.6309306934105321],
 'recall': [(1.0, 1.0), 1.0]}

In [16]:
classification_experiment.save_model_weights("linear-attention.weights")